<a href="https://colab.research.google.com/github/Khaled-Wael/NASA_Project/blob/YU/Nasa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

!pip install lightkurve astropy numpy scipy scikit-learn matplotlib
!git clone https://github.com/google-research/exoplanet-ml.git

Looking in indexes: https://download.pytorch.org/whl/cpu
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 98.0 MB/s eta 0:00:00
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11373 sha256=65e70982e69e06e0ebd825aa97a94f4adaff7c32055bef92714c956156b04a2c
  Stored in directory: /root/.cache/pip/wheels/04/15/cd/2f622795b09e83471a3be5d258

In [2]:
import os
import math
import numpy as np
from scipy.signal import savgol_filter
from scipy import stats
from sklearn.metrics import roc_auc_score, accuracy_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [5]:
def detrend_median(x, kernel_size=101):
# simple rolling-median detrend (works on 1D numpy arrays)
    if kernel_size % 2 == 0:
        kernel_size += 1
        pad = kernel_size // 2
        med = np.convolve(x, np.ones(kernel_size)/kernel_size, mode='same')
        return x - med

def savgol_detrend(x, window_length=101, polyorder=3):
      if window_length % 2 == 0:
            window_length += 1
      return x - savgol_filter(x, window_length=window_length, polyorder=polyorder)




def normalize(x, eps=1e-9):
    x = np.array(x, dtype=np.float32)
    med = np.median(x)
    mad = np.median(np.abs(x - med))
    return (x - med) / (mad + eps)




def phase_fold(time, flux, period, t0=0.0, width=None):
# returns phase-sorted flux centered on transit
    phase = ((time - t0 + 0.5 * period) % period) / period - 0.5
    order = np.argsort(phase)
    return phase[order], flux[order]

In [ ]:
hhhhh